In [27]:
import pandas as pd
import os
import numpy as np
import json
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union

In [28]:
# ディレクトリのパスを指定
csv_output_directory = '../output/nodes_data'  # ノード特徴量CSVファイルが含まれるディレクトリ
classified_output_directory = '../output/nodes_class_data'  # 結果を保存するディレクトリ
json_directory = '../rplan_json/shifted_external'  # JSONファイルのディレクトリ（部屋やドアのデータを含む）

# 出力ディレクトリを作成（存在しない場合）
os.makedirs(csv_output_directory, exist_ok=True)
os.makedirs(classified_output_directory, exist_ok=True)

In [29]:
# グリッドの設定
grid_unit = 5
max_coordinate = 225

# ROOM_CLASSのマッピング
ROOM_CLASS = {
    "living_room": 1,
    "kitchen": 2,
    "bedroom": 3,
    "bathroom": 4,
    "balcony": 5,
    "entrance": 6,
    "dining room": 7,
    "study room": 8,
    "storage": 10,
    "front door": 15,
    "unknown": 16,
    "interior_door": 17
}

room_type_to_node_class = {
    1: 'living_room',
    2: 'kitchen',
    3: 'bedroom',
    4: 'bathroom',
    5: 'balcony',
    6: 'entrance',
    7: 'dining_room',
    8: 'study_room',
    10: 'storage',
    15: 'front_door',
    17: 'interior_door'
}
default_node_class = 'empty'

In [30]:
# JSONディレクトリ内のすべてのファイルを処理
for filename in os.listdir(json_directory):
    if filename.endswith('.json'):
        # JSONファイルの読み込み
        json_file_path = os.path.join(json_directory, filename)
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        # 必要なデータを取得
        rooms_data = data['processed_data']['stage5_final_alignment']['rooms']
        doors_data = data['processed_data']['stage5_final_alignment']['doors']
        external_walls_data = data['processed_data']['stage5_final_alignment']['external_walls']

        # グリッドポイント（ノード）の生成
        nodes_list = []
        node_id = 0
        x_coords = np.arange(0, max_coordinate + grid_unit, grid_unit)
        y_coords = np.arange(0, max_coordinate + grid_unit, grid_unit)

        for x in x_coords:
            for y in y_coords:
                nodes_list.append({
                    'id': node_id,
                    'x': x,
                    'y': y,
                    'on_external_wall': 0,
                    'on_internal_wall': 0,
                    'outside_boundary': 0,
                    'inside_boundary': 0
                })
                node_id += 1

        # 部屋のポリゴンとそのクラスの対応を保持
        room_polygons_with_class = []
        for room in rooms_data:
            polygon = Polygon(room['polygon']['coordinates'][0])
            room_type = room['room_type']
            node_class = room_type_to_node_class.get(room_type, default_node_class)
            room_polygons_with_class.append({'polygon': polygon, 'class': node_class})

        # フロアプランの境界の作成
        floor_plan_boundary = unary_union([item['polygon'] for item in room_polygons_with_class])

        # 外壁のラインの作成
        external_wall_lines = [LineString([wall['p1'], wall['p2']]) for wall in external_walls_data]

        # ドアのラインとクラスの作成
        door_lines_with_class = []
        for door in doors_data:
            line = LineString([door['p1'], door['p2']])
            node_class = 'front_door' if door['type'] == 15 else 'interior_door'
            door_lines_with_class.append({'line': line, 'class': node_class})

        # 内壁のラインの作成（external_wall と door を除外）
        internal_wall_lines = []
        all_wall_lines = []
        for room in room_polygons_with_class:
            coords = list(room['polygon'].exterior.coords)
            for i in range(len(coords) - 1):
                line = LineString([coords[i], coords[i + 1]])
                all_wall_lines.append(line)

        # 重複しない壁のリストを作成
        unique_wall_lines = []
        for line in all_wall_lines:
            if not any(line.equals(other) or line.equals(LineString(list(other.coords)[::-1])) for other in unique_wall_lines):
                unique_wall_lines.append(line)

        # 外壁とドアに含まれない壁を内部壁とする
        for line in unique_wall_lines:
            is_external = any(line.equals(ext_line) for ext_line in external_wall_lines)
            is_door = any(line.equals(door_line['line']) for door_line in door_lines_with_class)
            if not is_external and not is_door:
                internal_wall_lines.append(line)

        # ノード特徴量の計算
        for node in nodes_list:
            point = Point(node['x'], node['y'])

            # 境界内外の判定
            if floor_plan_boundary.contains(point) or floor_plan_boundary.touches(point):
                node['inside_boundary'] = 1
                node['outside_boundary'] = 0
            else:
                node['inside_boundary'] = 0
                node['outside_boundary'] = 1

            # 外壁上かどうか
            node['on_external_wall'] = int(any(line.distance(point) < 1e-6 for line in external_wall_lines))
            # 内壁上かどうか（外壁でない場合にのみチェック）
            if node['on_external_wall'] == 0:
                node['on_internal_wall'] = int(any(line.distance(point) < 1e-6 for line in internal_wall_lines))

        # ノード特徴量をDataFrameに変換して保存
        nodes_df = pd.DataFrame(nodes_list)
        csv_output_file_path = os.path.join(csv_output_directory, f'{os.path.splitext(filename)[0]}_nodes.csv')
        nodes_df.to_csv(csv_output_file_path, index=False)

        # 各ノードにクラスを割り当てる
        nodes_df['node_class'] = default_node_class
        for index, node in nodes_df.iterrows():
            point = Point(node['x'], node['y'])
            
            # 1. 境界外なら 'empty' クラス
            if node['outside_boundary'] == 1:
                nodes_df.at[index, 'node_class'] = 'empty'
                continue

            # 2. ドア上にあるかどうか（優先）
            if any(door['line'].distance(point) < 1e-6 for door in door_lines_with_class):
                nodes_df.at[index, 'node_class'] = next(
                    door['class'] for door in door_lines_with_class if door['line'].distance(point) < 1e-6
                )
                continue

            # 3. 外壁上かどうか
            if node['on_external_wall'] == 1:
                nodes_df.at[index, 'node_class'] = 'exterior_wall'
                continue

            # 4. 内壁上かどうか
            if node['on_internal_wall'] == 1:
                nodes_df.at[index, 'node_class'] = 'interior_wall'
                continue

            # 5. 部屋内かどうか
            for room in room_polygons_with_class:
                if room['polygon'].contains(point) or room['polygon'].touches(point):
                    nodes_df.at[index, 'node_class'] = room['class']
                    break

        # クラス分類されたノードデータを保存
        classified_output_file_path = os.path.join(classified_output_directory, f'{os.path.splitext(filename)[0]}_classified.csv')
        nodes_df.to_csv(classified_output_file_path, index=False)
        print(f"Processed {filename}: saved node features to {csv_output_file_path} and classified nodes to {classified_output_file_path}")

Processed 10037_processed.json: saved node features to ../output/nodes_data\10037_processed_nodes.csv and classified nodes to ../output/nodes_class_data\10037_processed_classified.csv
Processed 10046_processed.json: saved node features to ../output/nodes_data\10046_processed_nodes.csv and classified nodes to ../output/nodes_class_data\10046_processed_classified.csv
Processed 1014_processed.json: saved node features to ../output/nodes_data\1014_processed_nodes.csv and classified nodes to ../output/nodes_class_data\1014_processed_classified.csv
Processed 10208_processed.json: saved node features to ../output/nodes_data\10208_processed_nodes.csv and classified nodes to ../output/nodes_class_data\10208_processed_classified.csv
Processed 10303_processed.json: saved node features to ../output/nodes_data\10303_processed_nodes.csv and classified nodes to ../output/nodes_class_data\10303_processed_classified.csv
Processed 10411_processed.json: saved node features to ../output/nodes_data\10411_p

c:\Users\student\anaconda3\envs\housegan-data-reader\lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Processed 21268_processed.json: saved node features to ../output/nodes_data\21268_processed_nodes.csv and classified nodes to ../output/nodes_class_data\21268_processed_classified.csv
Processed 21309_processed.json: saved node features to ../output/nodes_data\21309_processed_nodes.csv and classified nodes to ../output/nodes_class_data\21309_processed_classified.csv
Processed 21365_processed.json: saved node features to ../output/nodes_data\21365_processed_nodes.csv and classified nodes to ../output/nodes_class_data\21365_processed_classified.csv
Processed 21375_processed.json: saved node features to ../output/nodes_data\21375_processed_nodes.csv and classified nodes to ../output/nodes_class_data\21375_processed_classified.csv
Processed 21442_processed.json: saved node features to ../output/nodes_data\21442_processed_nodes.csv and classified nodes to ../output/nodes_class_data\21442_processed_classified.csv
Processed 21449_processed.json: saved node features to ../output/nodes_data\2144

c:\Users\student\anaconda3\envs\housegan-data-reader\lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Processed 21801_processed.json: saved node features to ../output/nodes_data\21801_processed_nodes.csv and classified nodes to ../output/nodes_class_data\21801_processed_classified.csv
Processed 21974_processed.json: saved node features to ../output/nodes_data\21974_processed_nodes.csv and classified nodes to ../output/nodes_class_data\21974_processed_classified.csv
Processed 21981_processed.json: saved node features to ../output/nodes_data\21981_processed_nodes.csv and classified nodes to ../output/nodes_class_data\21981_processed_classified.csv
Processed 21989_processed.json: saved node features to ../output/nodes_data\21989_processed_nodes.csv and classified nodes to ../output/nodes_class_data\21989_processed_classified.csv
Processed 21998_processed.json: saved node features to ../output/nodes_data\21998_processed_nodes.csv and classified nodes to ../output/nodes_class_data\21998_processed_classified.csv
Processed 22250_processed.json: saved node features to ../output/nodes_data\2225

...

Processed 76026_processed.json: saved node features to ../output/nodes_data\76026_processed_nodes.csv and classified nodes to ../output/nodes_class_data\76026_processed_classified.csv


c:\Users\student\anaconda3\envs\housegan-data-reader\lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Processed 76048_processed.json: saved node features to ../output/nodes_data\76048_processed_nodes.csv and classified nodes to ../output/nodes_class_data\76048_processed_classified.csv
Processed 76302_processed.json: saved node features to ../output/nodes_data\76302_processed_nodes.csv and classified nodes to ../output/nodes_class_data\76302_processed_classified.csv
Processed 7642_processed.json: saved node features to ../output/nodes_data\7642_processed_nodes.csv and classified nodes to ../output/nodes_class_data\7642_processed_classified.csv
Processed 76541_processed.json: saved node features to ../output/nodes_data\76541_processed_nodes.csv and classified nodes to ../output/nodes_class_data\76541_processed_classified.csv
Processed 7669_processed.json: saved node features to ../output/nodes_data\7669_processed_nodes.csv and classified nodes to ../output/nodes_class_data\7669_processed_classified.csv
Processed 76740_processed.json: saved node features to ../output/nodes_data\76740_proc

c:\Users\student\anaconda3\envs\housegan-data-reader\lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Processed 79295_processed.json: saved node features to ../output/nodes_data\79295_processed_nodes.csv and classified nodes to ../output/nodes_class_data\79295_processed_classified.csv
Processed 79343_processed.json: saved node features to ../output/nodes_data\79343_processed_nodes.csv and classified nodes to ../output/nodes_class_data\79343_processed_classified.csv
Processed 79429_processed.json: saved node features to ../output/nodes_data\79429_processed_nodes.csv and classified nodes to ../output/nodes_class_data\79429_processed_classified.csv
Processed 79573_processed.json: saved node features to ../output/nodes_data\79573_processed_nodes.csv and classified nodes to ../output/nodes_class_data\79573_processed_classified.csv
Processed 79670_processed.json: saved node features to ../output/nodes_data\79670_processed_nodes.csv and classified nodes to ../output/nodes_class_data\79670_processed_classified.csv
Processed 79674_processed.json: saved node features to ../output/nodes_data\7967